In [1]:
import requests
import time
import datetime

In [2]:
import pandas as pd

In [3]:
def sendMessage(para, mensaje):
    url = 'http://localhost:3001/lead'
    
    data = {
        "message": mensaje,
        "phone": para
    }
    headers = {
        'Content-Type':'application/json'
    }
    print(data)
    response = requests.post(url, json=data, headers=headers)
    time.sleep(60)
    return response

In [4]:
now = datetime.datetime.now()

In [5]:
mario = '59175380725'

In [79]:
def calcular_datos():
    df_playa = pd.read_excel(r'C:/Documents/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/ReportePlaya.xlsx')
    df_molienda = pd.read_excel(r'C:/Documents/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/Molienda.xlsx')
    df_horarios = pd.read_excel(r'C:/Documents/Ingenio Azucarero Guabira S.A/COOR_GERENCIA_CANA - Parte_Horarios/Horarios.xlsx')
    
    df_res_molienda = pd.merge(df_molienda, df_horarios[['Hora', 'Orden_Hora']], left_on='hora2', right_on='Hora', how='left')
    
    #cantidad de paquetes
    #cantidad de caña disponible
    filtro = df_playa[(df_playa['status'] == 'PL') | (df_playa['status'] == 'IN')]
    cantidad_paquetes = filtro['cantPqt'].sum()
    cana_disponible = cantidad_paquetes * 14
    
    #promedio lleganda pq
    df_playa['Hora_Entera'] = df_playa['horaDocum'].str[11:13].astype(int)
    max_hora_ent = df_playa['Hora_Entera'].max() - 3
    filtered_df = df_playa[df_playa['Hora_Entera'] >= max_hora_ent]
    sum_cant_pqt = filtered_df['cantPqt'].sum()
    promedio_llegada_pq = sum_cant_pqt / 3
    
    #trapiches
    # trapiche1    210 tn/ha    15 paquetes
    # trapiche2    690 tn/ha    49 paquetes
    
    #horas molienda
    horas_molienda_t1 = cantidad_paquetes / 15
    horas_molienda_t2 = cantidad_paquetes / 49
    horas_molienda_total = cantidad_paquetes / (15 + 49)
    
    #total paquetes resto dia
    total_paquetes_resto_dia_t1 = promedio_llegada_pq * horas_molienda_t1
    total_paquetes_resto_dia_t2 = promedio_llegada_pq * horas_molienda_t2
    total_paquetes_resto_dia_total = promedio_llegada_pq * horas_molienda_total
    
    #toneladas
    toneladas = df_molienda['netWeight'].sum() / 1000
    
    #planificacion actual
    planificacion_actual_t1 = df_res_molienda['Orden_Hora'].max() * 210
    planificacion_actual_t2 = df_res_molienda['Orden_Hora'].max() * 690
    planificacion_actual_total = df_res_molienda['Orden_Hora'].max() * (210 + 690)
    
    #diferencia actual
    diferencia_actual_t1 = toneladas - planificacion_actual_t1
    diferencia_actual_t2 = toneladas - planificacion_actual_t2
    diferencia_actual_total = toneladas - planificacion_actual_total
    
    #orden hora
    orden_hora = 24 - df_res_molienda['Orden_Hora'].max()
    
    #toneladas promedio
    toneladas_prom = (df_molienda['netWeight'].sum() / 1000) / (24 - orden_hora)
    
    mensaje = f'''
*REPORTE*
*🚛 Paquetes disponibles:* {round(cantidad_paquetes,2)}
*⚖️ Toneladas aprox.:* {round(cana_disponible,2)}
*⏱️ Promedio llegada pq.:* {round(promedio_llegada_pq,2)}
*📈 Paquetes estimados:* {round(total_paquetes_resto_dia_total,2)}
*🎋 Molienda actual:* {round(toneladas,2)}
*📅 Planificacion actual:* {round(planificacion_actual_total,2)}
*🔻 Diferencia actual:* {round(diferencia_actual_total,2)}
*🕒 Promedio horario:* {round(toneladas_prom,2)}
'''
    return mensaje

In [80]:
m = calcular_datos()

In [81]:
print(m)


*REPORTE*
*🚛 Paquetes disponibles:* 83
*⚖️ Toneladas aprox.:* 1162
*⏱️ Promedio llegada pq.:* 36.33
*📈 Paquetes estimados:* 47.12
*🎋 Molienda actual:* 9272.03
*📅 Planificacion actual:* 21600
*🔻 Diferencia actual:* -12327.97
*🕒 Promedio horario:* 386.33



In [ ]:
while True:
    #now = datetime.datetime.now()
    cantidad_paquetes, cana_disponible = calcular_datos()
    mensaje = f'''
*Cantidad de paquetes en playa:* {cantidad_paquetes} paquetes.
*Cantidad de caña disponible:* {cana_disponible} toneladas.
'''
    sendMessage(me, mensaje)

{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
{'message': '\n*Cantidad de paquetes en playa:* 102 paquetes.\n*Cantidad de caña disponible:* 1428 toneladas.\n', 'phone': '59178194371'}
